In [2]:
#Course_code
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

In [ ]:
osm_file = open("chicago_abbrev.osm","r")

In [ ]:
street_type_re = re.compile(r'\S+\.?$',re.IGNORECASE) #extract street info.
street_types = defaultdict(int)

In [ ]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] +=1

In [ ]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys,key=lambda s:s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" %(k, v)

In [ ]:
def is_street_name(elem):
    return (elem.tag =="tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)
    
if __name__ =='__main__':
    audit()

In [ ]:
#auditing a cross_field constraint
def ensure_float(v):
    if is_numnber(v):
        return float(v)
    
def audit_population_density(input_file):
    for row in input_file:
        population = ensure_float(row['populationTotal'])
        area = ensure_float(row['areaLand'])
        population_density = ensure_float(row['populationDensity'])
        if population and area and population_density:
            calculated_density = population / area
            if math.fabs(calculated_density - population_density) >10:
                print "Possibly bad population density for ", row['name']
if __name__=='__main__':
    input_file = csv.DictReader(open("cities.csv"))
    skip_lines(input_file, 3)
    audit_population_density(input_file)